In [311]:
!pip install lime --user
!pip install tabulate --user
!pip install pdfkit --user

from sklearn import model_selection
from sklearn import ensemble
from sklearn import datasets

from lime import lime_tabular

import os
from multiprocessing import Process
import pandas as pd
import numpy as np
import sklearn
import lime
import pickle

PICKLES_PATH = "/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/classification/pickles"
SUBSETS_PATH = "/afs/inesc-id.pt/home/aof/thesis/subsets"
EXPLANATIONS_PATH = "/afs/inesc-id.pt/home/aof/public_html"

BOOTSTRAP = '<head><link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous"></head>'
#=================================================================================
'''
iris = datasets.load_iris()

train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)

explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=iris.feature_names, class_names=iris.target_names, discretize_continuous=True)

for i in range(test.shape[0]):
    exp = explainer.explain_instance(test[i], rf.predict_proba, num_features=3, top_labels=2)
    print(vars(exp))
    exp.show_in_notebook(show_table=True, show_all=True)   
    #break
    print(vars(exp.domain_mapper))
    break
'''

thinc 6.10.2 requires msgpack-python, which is not installed.
tensorflow 1.15.2 requires google-pasta>=0.1.6, which is not installed.
tensorflow 1.15.2 requires grpcio>=1.8.6, which is not installed.
tensorflow 1.15.2 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 1.15.2 requires tensorboard<1.16.0,>=1.15.0, which is not installed.
tensorflow 1.15.2 requires tensorflow-estimator==1.15.1, which is not installed.
spacy 2.0.5 requires msgpack-python, which is not installed.
texar 0.2.0 requires funcsigs, which is not installed.
python-keyczar 0.71h requires pycrypto>2.0, which is not installed.
thinc 6.10.2 has requirement cytoolz<0.9,>=0.8, but you'll have cytoolz 0.10.1 which is incompatible.
thinc 6.10.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.5 which is incompatible.
tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
tensorflow 1.15.2 has requirement wrapt>=1.11.1, but you'll have wrapt 1.10.

'\niris = datasets.load_iris()\n\ntrain, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)\n\nrf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)\nrf.fit(train, labels_train)\n\nexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=iris.feature_names, class_names=iris.target_names, discretize_continuous=True)\n\nfor i in range(test.shape[0]):\n    exp = explainer.explain_instance(test[i], rf.predict_proba, num_features=3, top_labels=2)\n    print(vars(exp))\n    exp.show_in_notebook(show_table=True, show_all=True)   \n    #break\n    print(vars(exp.domain_mapper))\n    break\n'

In [312]:
def write_html_to_file(html, path):
    f = open(path, "w")
    f.write(html)
    f.close()

In [313]:
'''
    Model: MLPClassifier instance
    Patient_Lines: lines of CSV corresponding to the patient to be tested
    Train: Training fractionf of the CSV
    Feature_names: list of features (columns of csv)
    Output path: file path to write the output, None for writing on screen
'''
def explain_patient(model, patient_lines, train, feature_names, output_path=None):
    
    num_features = 5
    
    #Build tabular
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=feature_names, class_names=['HC', 'PD'], discretize_continuous=True)

    # Pass each line to LimeTabularExplainer
    explanations = []
    for i, row in patient_lines.iterrows():
        #Run only 1 in every 10 time frames (10 samples / second)
        if i % 10 == 0:
            explanations.append(explainer.explain_instance(row, model.predict_proba, num_features=num_features, top_labels=1))

    # Sum all local_exp
    total_local_exp = {}
    for explanation in explanations:
        #local_exp = {1: [(12, 0.139704949653412), (44, 0.10278682378094679)]}
        local_exp = explanation.local_exp
        
        # will produce {<feature>: (count, sum_weight), ...
        for tup in local_exp[0 if 0 in local_exp else 1]:
            #tup = (12, 0.139704949653412)
            feature = feature_names[tup[0]]
            if feature in total_local_exp:
                total_local_exp[feature][0] += 1
                total_local_exp[feature][1] += tup[1]
            else:
                total_local_exp[feature] = [1, tup[1]]
    
    #Average local_exp
    # Key --> feature
    # value --> average value
    averaged_local_exp = {}

    for key in total_local_exp.keys():
        tup = total_local_exp[key]
        averaged_local_exp[key] = tup[1] / tup[0]

    # Normalize local_exp
    '''total_sum = np.float64(sum(averaged_local_exp)) 
    for key in averaged_local_exp.keys():
        averaged_local_exp[key] /= total_sum
    '''
    # Normalize predict_proba
    average_predict_proba = [sum(items) for items in zip(*[explanation.predict_proba for explanation in explanations ])]
    average_predict_proba /= np.float64(sum(average_predict_proba))

    display_or_save_patient(averaged_local_exp, average_predict_proba, class_names, output_path, num_features)


In [314]:
def generate_suffix(params):
    return "{}_{}_{}_{}".format(params['solver'], params['alpha'], params['max_iter'], params['activation'])

In [323]:
from IPython.display import HTML, display
import tabulate

def display_or_save_patient(averaged_local_exp, average_predict_proba, class_names, output_path, num_features):

    prediction_proba = \
            '<table class="table" style="margin: 0 auto;"> \
                <thead> \
                    <tr> \
                        <td style="margin: 2px; text-align:center; font-weight: bold;">HC</td> \
                        <td style="margin: 2px; text-align:center; font-weight: bold;">PD</td> \
                    </tr> \
                </thead> \
                <tr> \
                    <td style="margin: 2px; text-align: center;">{}</td> \
                    <td style="margin: 2px; text-align: center;">{}</td> \
                </tr> \
            </table>' \
            .format("{:.2f}".format(average_predict_proba[0]), "{:.2f}".format(average_predict_proba[1]))
    
    averaged_local_exp = {k: v for k, v in sorted(averaged_local_exp.items(), key=lambda item: -abs(item[1]))}

    feature_weights = averaged_local_exp.items()
    feature_weight_table = '<table class="table" style="margin: 0 auto;">'

    accounted = 0
    for feature_weight in feature_weights:
        s = '<tr> \
                <td style="margin: 2px; text-align:center; font-weight: bold;">{}</td> \
                <td style="margin: 2px; text-align:center;">{}</td> \
            </tr>' \
            .format(feature_weight[0], feature_weight[1])
        feature_weight_table += s
        
        accounted += 1
        if accounted == num_features:
            break

    feature_weight_table += '</table>'
    
    #for feature_index in averaged_local_exp:
    #    feature_weight.append([feature_index, averaged_local_exp[feature_index]])
    
    
    table = '<table class="table" style="margin: 0 auto;"> \
                <thead> \
                    <tr> \
                        <td style="width: 45%; margin: 2px; text-align:center; font-weight: bold;">Prediction Probabilities</td> \
                        <td style="width: 45%; margin: 2px; text-align:center; font-weight: bold;">Feature weight</td> \
                    </tr> \
                </thead> \
                <tr> \
                    <td style="width: 45%; margin: 2px; text-align:center">{}</td> \
                    <td style="width: 45%; margin: 2px; text-align:center">{}</td> \
                </tr> \
            </table>' \
        .format(prediction_proba, feature_weight_table)

    if output_path == None:
        display(HTML(table))
    else:
        write_html_to_file(BOOTSTRAP + table, output_path)

In [316]:
def generate_mlp_params_list(test=False):
    result = []
    alphas = [0.0001, 0.001, 0.01]
    max_iters = [2000, 5000]
    solvers = ['lbfgs', 'adam']

    if test:
        return [{'alpha': 0.001, 'max_iter': 1, 'solver': 'adam', 'activation': 'tanh'}]

    for alpha in alphas:
        for max_iter in max_iters:
            for solver in solvers:
                result.append({
                    "alpha": alpha,
                    "max_iter": max_iter,
                    "activation":  "tanh",
                    "solver": solver
                })
    return result

In [317]:
def explain(train_path, test_path, experiment, dataset, params, dry_run=False):
    pickle_file = '{}/{}/{}/{}.pkl'.format(PICKLES_PATH, experiment, dataset, generate_suffix(params))
    if os.path.isfile(pickle_file):
            with open(pickle_file, 'rb') as filename:
                model = pickle.load(filename)
    else:
        print("WARNING: No pickle found @ {}. Skipping...".format(pickle_file))
        return

    train = pd.read_csv(train_path, sep=";")
    test  = pd.read_csv(test_path , sep=";")
        
    train = train[train.columns.difference(['frameTime', 'label', 'name'])]

    patients = list(set(test['name']))
    feature_names = [feature for feature in list(test.columns) if feature not in ['label', 'name', 'frameTime']]

    for patient in patients:
        patient_lines = test[test["name"] == patient]
        patient_lines = patient_lines[patient_lines.columns.difference(['frameTime', 'label', 'name'])]
        
        output_path = "{}/{}/{}/{}_{}.html".format(EXPLANATIONS_PATH, experiment, dataset, patient.split('_')[0], generate_suffix(params))
        
        if os.path.isfile(output_path):
            print("Explanation exists, ignoring...")
            return

        print("Explaining patient {} to {}".format(patient.split('_')[0], output_path))
        if not dry_run:
            explain_patient(model, patient_lines, train.to_numpy(), feature_names, output_path=output_path)

In [318]:
def get_train_test_path(experiment, dataset, params):
    if 'independent' in experiment:
        train = "{}/{}/{}/{}_train.csv".format(SUBSETS_PATH, experiment, dataset, generate_suffix(params))
        test =  "{}/{}/{}/{}_test.csv".format (SUBSETS_PATH, experiment, dataset, generate_suffix(params))        
    else:
        train = "{}/{}/{}/{}_train.csv".format(SUBSETS_PATH, experiment, dataset, generate_suffix(params))
        test =  "{}/{}/{}/{}_test.csv".format (SUBSETS_PATH, experiment, dataset, generate_suffix(params))

    return train, test

In [319]:
def get_datasets_names():
    return {
        'baseline': ['fralusopark', 'gita', 'mdvr_kcl'],
        'baseline_200': ['fralusopark', 'gita', 'mdvr_kcl'],
        'independent': ['fralusopark', 'gita', 'mdvr_kcl'],
        'independent_200': ['fralusopark', 'gita', 'mdvr_kcl'],
        'semi': ['fralusopark_gita', 'fralusopark_mdvr_kcl', 'gita_fralusopark', 'gita_mdvr_kcl', 'mdvr_kcl_fralusopark', 'mdvr_kcl_gita'],
        'semi_200': ['fralusopark_gita', 'fralusopark_mdvr_kcl', 'gita_fralusopark', 'gita_mdvr_kcl', 'mdvr_kcl_fralusopark', 'mdvr_kcl_gita']
    }

In [320]:
def run_experiment_patients(experiment, mlp_params, dataset_names):
    for params in mlp_params:
        for dataset in dataset_names[experiment]:
            train_path, test_path = get_train_test_path(experiment, dataset, params)

            explain(train_path, test_path, experiment, dataset, params)


In [322]:
def main():
    experiments = ['baseline', 'baseline_200'] # 'semi', 'semi_200'] # 'independent', 'independent_200']
    mlp_params = generate_mlp_params_list()
    dataset_names = get_datasets_names()
    
    experiment_processes = []
    for experiment in experiments:
        experiment_processes.append(Process(target=run_experiment_patients, args=(experiment, mlp_params, dataset_names,)))

    for experiment_process in experiment_processes:
        experiment_process.start()

    for experiment_process in experiment_processes:
        experiment_process.join()


In [293]:
'''def main():
    experiments = ['baseline'] #, 'baseline_200', 'semi', 'semi_200'] # 'independent', 'independent_200']
    mlp_params = generate_mlp_params_list()
    dataset_names = get_datasets_names()
    
    for experiment in experiments:
        for params in mlp_params:
            for dataset in dataset_names[experiment]:
                train_path, test_path = get_train_test_path(experiment, dataset, params)
                
                explain(train_path, test_path, experiment, dataset, params)
                
                break
            break
        break'''

In [ ]:
if __name__ == '__main__':
    main()

Explanation exists, ignoring...
Explaining patient AVPEPUDEAC0053 to /afs/inesc-id.pt/home/aof/public_html/baseline/gita/AVPEPUDEAC0053_lbfgs_0.0001_2000_tanh.html
Explaining patient 283 to /afs/inesc-id.pt/home/aof/public_html/baseline_200/fralusopark/283_lbfgs_0.0001_2000_tanh.html
